In [1]:
import requests
import webbrowser
from bs4 import BeautifulSoup
#import ast
#import re
import pandas as pd
import numpy as np

In [160]:
url = 'https://www.ptt.cc/bbs/creditcard/M.1553828664.A.F07.html'
r = requests.get(url)
#print(r.url)
#webbrowser.open(r.url)

In [161]:
soup = BeautifulSoup(r.text,'lxml')

In [162]:
tags = [n.text.strip(' ') for n in soup.select('div.push span.push-tag')]

In [163]:
authors = [n.text.strip(' ') for n in soup.select('div.push span.push-userid')]

In [164]:
contents = [n.text.strip(': ').strip(' ') for n in soup.select('div.push span.push-content')]

In [165]:
dts = [n.text.strip('\n').strip(' ') for n in soup.select('div.push span.push-ipdatetime')]

In [166]:
title = soup.select('meta')[6]['content']
titles = [title] * len(dts)
urls   = [url]   * len(dts)

In [167]:
#重整資料為DataFrame
df = pd.DataFrame(np.column_stack([authors, dts, tags, contents, titles, urls]), 
                               columns=['作者','日期時間','情緒','輿情內容','主文標題',
                                        '文章連結'])
df

,作者,日期時間,情緒,輿情內容,主文標題,文章連結
0,laimingtse,03/29 11:06,推,須要年費嗎？,[心得] 北富邦加辦OMIYAGE卡已收到,https://www.ptt.cc/bbs/creditcard/M.1553828664...
1,b731863888,03/29 11:07,推,當天辦昨天才跳出審核資訊還查不到卡號跟核卡資訊,[心得] 北富邦加辦OMIYAGE卡已收到,https://www.ptt.cc/bbs/creditcard/M.1553828664...
2,peiningyu,03/29 11:11,推,等寄來 你的好快,[心得] 北富邦加辦OMIYAGE卡已收到,https://www.ptt.cc/bbs/creditcard/M.1553828664...
3,william7713,03/29 11:11,→,我也在等寄來 下周出國感覺來不及了...,[心得] 北富邦加辦OMIYAGE卡已收到,https://www.ptt.cc/bbs/creditcard/M.1553828664...
4,mowoco,03/29 11:11,推,也太快！,[心得] 北富邦加辦OMIYAGE卡已收到,https://www.ptt.cc/bbs/creditcard/M.1553828664...
5,koharapei,03/29 11:23,推,對我而言line point等同現金，那跟上海一樣都是3%，這,[心得] 北富邦加辦OMIYAGE卡已收到,https://www.ptt.cc/bbs/creditcard/M.1553828664...
6,koharapei,03/29 11:23,→,張又是jcb，應該小贏一些些吧？可能只差在jcb刷卡店家,[心得] 北富邦加辦OMIYAGE卡已收到,https://www.ptt.cc/bbs/creditcard/M.1553828664...
7,koharapei,03/29 11:23,→,少？,[心得] 北富邦加辦OMIYAGE卡已收到,https://www.ptt.cc/bbs/creditcard/M.1553828664...
8,gyu67,03/29 11:23,推,收到+1,[心得] 北富邦加辦OMIYAGE卡已收到,https://www.ptt.cc/bbs/creditcard/M.1553828664...
9,Arthur818,03/29 11:31,→,富邦效率最近不錯高,[心得] 北富邦加辦OMIYAGE卡已收到,https://www.ptt.cc/bbs/creditcard/M.1553828664...


In [168]:
from datetime import datetime

##PTT推文清整
def merge_comments(data):
    r=0
    while r < len(data)-1:
        while r+1 <= len(data)-1:
            if data.iloc[r+1,0]==data.iloc[r,0] and (datetime.strptime(data.iloc[r+1,1],'%m/%d %H:%M') - datetime.strptime(data.iloc[r,1],'%m/%d %H:%M')).total_seconds() <= 120 and data.iloc[r+1,2]=='→':
                data.iloc[r,3] = data.iloc[r,3]+''.join(data.iloc[r+1,3])
                data = data.drop([r+1]).reset_index().iloc[:,1:]
            else:
                break
        while r+2 <= len(data)-1:
            if data.iloc[r+2,0]==data.iloc[r,0] and (datetime.strptime(data.iloc[r+2,1],'%m/%d %H:%M') - datetime.strptime(data.iloc[r,1],'%m/%d %H:%M')).total_seconds() <= 120 and data.iloc[r+2,2]=='→':
                data.iloc[r,3] = data.iloc[r,3]+''.join(data.iloc[r+2,3])
                data = data.drop([r+2]).reset_index().iloc[:,1:]
            else:
                break
        while r+3 <= len(data)-1:
            if data.iloc[r+3,0]==data.iloc[r,0] and (datetime.strptime(data.iloc[r+3,1],'%m/%d %H:%M') - datetime.strptime(data.iloc[r,1],'%m/%d %H:%M')).total_seconds() <= 120 and data.iloc[r+3,2]=='→':
                data.iloc[r,3] = data.iloc[r,3]+''.join(data.iloc[r+3,3])
                data = data.drop([r+3]).reset_index().iloc[:,1:]
            else:
                break
        while r+4 <= len(data)-1:
            if data.iloc[r+4,0]==data.iloc[r,0] and (datetime.strptime(data.iloc[r+4,1],'%m/%d %H:%M') - datetime.strptime(data.iloc[r,1],'%m/%d %H:%M')).total_seconds() <= 120 and data.iloc[r+4,2]=='→':
                data.iloc[r,3] = data.iloc[r,3]+''.join(data.iloc[r+4,3])
                data = data.drop([r+4]).reset_index().iloc[:,1:]
            else:
                break
        while r+5 <= len(data)-1:
            if data.iloc[r+5,0]==data.iloc[r,0] and (datetime.strptime(data.iloc[r+5,1],'%m/%d %H:%M') - datetime.strptime(data.iloc[r,1],'%m/%d %H:%M')).total_seconds() <= 120 and data.iloc[r+5,2]=='→':
                data.iloc[r,3] = data.iloc[r,3]+''.join(data.iloc[r+5,3])
                data = data.drop([r+5]).reset_index().iloc[:,1:]
            else:
                break
        r += 1 
    return data

In [169]:
df2 = merge_comments(df)

In [170]:
df2.to_csv('D:/brown.lai/Documents/Social Listening Project/JCB/ptt.csv')